In [127]:
from videoModel import getModel
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML
from matplotlib import animation, rc
import matplotlib.image as mpimg
import glob
import sys
from keras.preprocessing import image
from utils import *
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import os
import re

datasetRoot = "/home/ubuntu/CourseAI/datasets/facs/"


In [61]:
weights_path = "weights/emotions_clas.hdf5"

model_emotions = Sequential()
model_emotions.add(Dense(32, activation='sigmoid', input_dim=65))
model_emotions.add(Dense(7, activation='softmax'))
model_emotions.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
if not os.path.exists(weights_path):
    model_emotions.fit(X, Y, epochs=1000, batch_size=128,callbacks=[ModelCheckpoint(weights_path)])
else :
    model_emotions.load_weights(weights_path)

In [177]:
emotion_values={3:"disgust",7:"surprised",4:"fear",5:"happy",6:"sad",1:"contempt",2:"contempt"}
emotion_dict=emotion_mapping()

In [178]:
def images_without_emotions():
    pattern=re.compile(r"S\d{3}/\d{3}")
    emotions=glob.glob(datasetRoot+"emotions/*/*")
    withoutEmotions = [pattern.findall(personID)[0] for index,personID in enumerate(emotions)
                       if len(glob.glob(emotions[index]+"/*"))==0]
    return withoutEmotions


In [35]:
def emotion_mapping():
    emotions = glob.glob(datasetRoot+"emotions/*/*/*.npy")
    
    pattern=re.compile(r"S\d{3}/\d{3}")
    emotion_dict={pattern.findall(file)[0]:np.load(file) for file in emotions}
    return emotion_dict
    

In [2]:
seq_length = 16
# In[22]:
rc('animation', html='html5')
def plot_movie_mp4(imgSeqPath):
#With ful list of pathes create animation
    imagearray=[mpimg.imread(path) for path in glob.glob(imgSeqPath + '/*.png')]
    dpi = 100.0
    xpixels, ypixels = imagearray[0].shape[0], imagearray[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(imagearray[0],cmap='gray')
    plt.close()
    def animate(i):
        im.set_array(imagearray[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(imagearray))
    plt.show()
    return anim

In [3]:
def getPreprocessedFrames(frame_paths, frames_num):
    output_imgs = np.zeros((1, frames_num, 224, 224, 3))
    for i in range(frames_num):
        img = image.load_img(frame_paths[i], target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        output_imgs[0][i] = x
    return output_imgs

def extractLandmarks(land_paths, lands_num):
    output_lands = np.zeros((1, lands_num, 68, 2))
    for i in range(lands_num):
        output_lands[0][i] = np.load(land_paths[i])
    return output_lands
    


In [198]:
def predict_func(personID,topn=7,threshold = 0.1):
    
    imageFramesPath= datasetRoot+"images/"+personID
    landmarksPath=datasetRoot+"landmarks/"+personID
    frame_paths = glob.glob(imageFramesPath + '/*.png')
    land_paths = glob.glob(landmarksPath + '/*.npy')
    frame_num = min(seq_length, len(frame_paths))
    model = getModel(frameCount=frame_num, nb_classes=65)
    model.load_weights('./weights/facsModel.hdf5')
    preds = model.predict([getPreprocessedFrames(frame_paths, frame_num), extractLandmarks(land_paths, frame_num)])[0]    
    confident_preds = preds[np.where(preds>threshold)]
    top_indices = preds.argsort()[::-1][:topn]
    encoded = np.zeros(65)
    encoded[top_indices]=1
    encoded=np.array(encoded).reshape(-1,65)
    prediction_emotion=np.argmax(model_emotions.predict(encoded))
    print("Predicted emotion is "+ emotion_values[prediction_emotion+1].upper())
    result = [['Prediction is action unit {}'.format(i), preds[i]] for i in top_indices]
    print(result)
    
    return plot_movie_mp4(imageFramesPath)
  
  


In [199]:
test_set=sorted(images_without_emotions())

In [206]:
predict_func(test_set[10])

Predicted emotion is FEAR
[['Prediction is action unit 25', 0.92970937], ['Prediction is action unit 20', 0.53220534], ['Prediction is action unit 4', 0.48666885], ['Prediction is action unit 7', 0.35252777], ['Prediction is action unit 12', 0.16813302], ['Prediction is action unit 11', 0.15629636], ['Prediction is action unit 1', 0.12483419]]


In [207]:
predict_func(test_set[1])

Predicted emotion is SAD
[['Prediction is action unit 4', 0.56024933], ['Prediction is action unit 17', 0.55223864], ['Prediction is action unit 1', 0.36931163], ['Prediction is action unit 2', 0.27343854], ['Prediction is action unit 15', 0.26758063], ['Prediction is action unit 25', 0.24529155], ['Prediction is action unit 7', 0.13760425]]


In [208]:
predict_func(test_set[2])

Predicted emotion is DISGUST
[['Prediction is action unit 25', 0.55068934], ['Prediction is action unit 4', 0.50752777], ['Prediction is action unit 17', 0.37968397], ['Prediction is action unit 7', 0.31770253], ['Prediction is action unit 9', 0.17526169], ['Prediction is action unit 6', 0.14880896], ['Prediction is action unit 15', 0.11875855]]


In [209]:
predict_func(test_set[5])

Predicted emotion is HAPPY
[['Prediction is action unit 25', 0.68905878], ['Prediction is action unit 1', 0.50094628], ['Prediction is action unit 2', 0.30100372], ['Prediction is action unit 12', 0.26086792], ['Prediction is action unit 16', 0.25106654], ['Prediction is action unit 15', 0.1657812], ['Prediction is action unit 5', 0.1239934]]
